<a href="https://colab.research.google.com/github/gl-24/my-NLP/blob/main/day-9/Fake_news_classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/competitions/fake-news/data"
)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gunjanlunkad
Your Kaggle Key: ··········


100%|██████████| 46.5M/46.5M [00:01<00:00, 28.6MB/s]



Extracting archive ./fake-news/fake-news.zip to ./fake-news


In [4]:
%cd fake-news

/content/fake-news


In [1]:
import pandas as pd

In [5]:
df=pd.read_csv('train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
###Drop Nan Values
df=df.dropna()


In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [13]:
## Get the Dependent features
y=df['label']

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.8.2'

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [19]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [20]:
messages=X.copy()

In [21]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [22]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [31]:
messages.reset_index(inplace=True)

In [32]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [36]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [37]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [38]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4991, 1332, 2669, 2823, 4788, 595, 3503, 666, 4380, 4407],
 [2940, 47, 217, 4120, 4035, 4281, 2328],
 [1497, 3873, 3301, 99],
 [3949, 1298, 4093, 3974, 1968, 2167],
 [3914, 4035, 2941, 3722, 4704, 435, 4035, 2129, 1655, 843],
 [4239,
  1419,
  2487,
  3501,
  3927,
  240,
  4252,
  3804,
  4956,
  2824,
  2871,
  4672,
  4370,
  4817,
  2328],
 [4533, 2081, 4745, 30, 1443, 4553, 2680, 2725, 2544, 3859, 1332],
 [135, 1897, 3655, 2136, 2737, 370, 240, 4063, 2544, 3859, 1332],
 [2273, 4500, 2515, 3442, 3315, 3943, 1564, 1725, 240, 4637],
 [4107, 4515, 4836, 864, 376, 832, 146, 240],
 [3118, 4583, 716, 4243, 1822, 4209, 3736, 4944, 967, 1880, 1223],
 [3974, 3626, 4788, 3943, 240, 2737],
 [2059, 223, 341, 3154, 2007, 3232, 1718, 1098, 3435],
 [3375, 1780, 965, 1845, 2932, 4909, 255, 2544, 3859, 1332],
 [1270, 1578, 963, 3804, 2193, 2544, 3859, 1332],
 [4225, 647, 1887, 4546, 1356, 1559, 4646, 955, 2159, 4595],
 [1204, 2263, 47],
 [4670, 4972, 2191, 3502, 240, 1527, 4814, 2328],
 [4809, 14

In [39]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [40]:
onehot_repr[1]

[2940, 47, 217, 4120, 4035, 4281, 2328]

### Embedding Representation

In [43]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4991 1332 2669 ...    0    0    0]
 [2940   47  217 ...    0    0    0]
 [1497 3873 3301 ...    0    0    0]
 ...
 [2465 2641  645 ...    0    0    0]
 [ 819 2737 4887 ...    0    0    0]
 [3957 2565 2660 ...    0    0    0]]


In [44]:
embedded_docs[1]

array([2940,   47,  217, 4120, 4035, 4281, 2328,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [45]:
embedded_docs[0]

array([4991, 1332, 2669, 2823, 4788,  595, 3503,  666, 4380, 4407,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [46]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
len(embedded_docs),y.shape

(18285, (18285,))

In [48]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [49]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [51]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 9ms/step - loss: 0.3322 - accuracy: 0.8403 - val_loss: 0.2030 - val_accuracy: 0.9143
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1442 - accuracy: 0.9467 - val_loss: 0.2047 - val_accuracy: 0.9190
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0959 - accuracy: 0.9671 - val_loss: 0.2412 - val_accuracy: 0.9178
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0627 - accuracy: 0.9788 - val_loss: 0.2793 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0472 - accuracy: 0.9846 - val_loss: 0.3226 - val_accuracy: 0.9062
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0319 - accuracy: 0.9913 - val_loss: 0.3687 - val_accuracy: 0.9084
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0276 - accuracy: 0.9920 - val_loss: 0.4133 - val_accuracy: 0.9100
Epoch 

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [52]:
y_pred=model.predict(X_test)

In [53]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [54]:
from sklearn.metrics import confusion_matrix

In [55]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 305, 2311]])

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9009113504556753

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3419
           1       0.89      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

